In [2]:
import json
import requests
import pandas as pd 
import time

ModuleNotFoundError: No module named 'requests'

In [ ]:
df = pd.read_csv("/home/cristina/Projetos/cnes-enriquecimento-dados/CNES/data/cnes_amostra.csv", encoding="latin1", low_memory=False,nrows=50)

In [ ]:
def populate_cep_viacep(dataframe):
    dataframe_vazio = pd.DataFrame()
    cache = {}

    for row in dataframe.itertuples():
        cep = str(row.CO_CEP)

        # Remove traços e espaços se tiver
        cep = cep.replace("-", "").strip()

        # Ignora CEPs nulos ou muito curtos
        if pd.isna(cep) or len(cep) < 8:
            continue

        if cep not in cache:
            try:
                response = requests.get(f'https://viacep.com.br/ws/{cep}/json/', timeout=5)
                response.raise_for_status()
                data = response.json()

                # Se a API disser que o CEP não existe
                if "erro" in data:
                    print(f"CEP inválido: {cep}")
                    continue

                # Salva no cache como DataFrame
                cache[cep] = pd.json_normalize(data)
                time.sleep(0.5)  # pequena pausa para evitar bloqueio
            except Exception as e:
                print(f"Erro ao consultar {cep}: {e}")
                continue

        dataframe_vazio = pd.concat([dataframe_vazio, cache[cep]], ignore_index=True)

    return dataframe_vazio

In [ ]:
df_ceps = cep_api_viacep(df)

In [ ]:
df_ceps["cep"] = df_ceps["cep"].str.replace("-", "")

In [ ]:
df['CO_CEP'] = df['CO_CEP'].astype(str).str.zfill(8)

In [ ]:
df_enriquecido = df.merge(df_ceps, left_on='CO_CEP', right_on='cep', how='left')